In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
indivs_commits = pd.concat([pd.read_csv("data/merged_data/committers_info_pr.csv", index_col = 0),
                            pd.read_csv("data/merged_data/committers_info_push.csv", index_col = 0)])

In [3]:
event_types = np.unique([ele[:-16] for ele in os.listdir('data/github_clean/github_data_pre_18')])

In [4]:
def readFile(prefix):
    file_group1 = [file for file in os.listdir('data/github_clean/github_data_pre_18') if prefix in file]
    all_files1 = pd.concat([pd.read_csv(f'data/github_clean/github_data_pre_18/{f}', index_col = 0) for f in file_group1])
    file_group2 = [file for file in os.listdir('data/github_clean/filtered_github_data_large') if prefix in file]
    all_files2 = pd.concat([pd.read_csv(f'data/github_clean/github_data_pre_18/{f}', index_col = 0) for f in file_group2])
    all_files = pd.concat([all_files1, all_files2])

    all_files_grouped = all_files.groupby(['actor_id', 'actor_login', 'repo_id', 'repo_name', 'org_id','org_login']).agg({'created_at':[min, max, 'count']})
    setcols = ['actor_id', 'actor_login', 'repo_id', 'repo_name', 'org_id','org_login']
    setcols.extend([f'earliest_{prefix}', f'latest_{prefix}', f'count_{prefix}'])
    all_files_grouped.reset_index(inplace = True)
    all_files_grouped.columns = setcols
    return all_files_grouped

In [5]:
df_merged = readFile(event_types[0])
for event in event_types[1:]:
    if event not in ['forkEvent', 'watchEvent']:
        print(event)
        df_fixed = readFile(event)
        df_merged = pd.merge(df_merged, df_fixed, how = 'outer')

deleteEvent
issueCommentEvent
issuesEvent
prEvent
prEventCommits
prReviewCommentEvent
prReviewEvent
pushEvent
releaseEvent


In [6]:
df_actors = df_merged[['actor_id', 'actor_login']].drop_duplicates()
df_actors['type'] = 'general contributor'
df_actors.reset_index(inplace = True, drop = True)

In [7]:
df_fork = readFile('forkEvent')
df_merged_fork = pd.merge(df_merged, df_fork, how = 'outer')

In [8]:
df_actors_fork = df_merged_fork[['actor_id', 'actor_login']].drop_duplicates()
df_actors_fork = pd.merge(df_actors_fork, df_actors, how = 'left')

In [9]:
logins = indivs_commits[['actor_login']].dropna().drop_duplicates()
logins = logins[~logins['actor_login'].isin(df_actors_fork['actor_login'].tolist())]
logins['type'] = 'general contributor'
logins['actor_id'] = np.nan

In [10]:
df_actors_fork = pd.concat([df_actors_fork, logins])

In [11]:
df_actors_fork['type'] = df_actors_fork['type'].apply(lambda x: 'forker' if pd.isnull(x) else x)
#df_actors_fork.to_csv('data/inputs/all_contributors.csv')

In [12]:
df_merged_fork

,actor_id,actor_login,repo_id,repo_name,org_id,org_login,earliest_createEvent,latest_createEvent,count_createEvent,earliest_deleteEvent,...,count_prReviewEvent,earliest_pushEvent,latest_pushEvent,count_pushEvent,earliest_releaseEvent,latest_releaseEvent,count_releaseEvent,earliest_forkEvent,latest_forkEvent,count_forkEvent
0,124,brosner,39740335,kelproject/pykube,17834534.0,kelproject,2016-05-21 16:11:14+00:00,2017-03-06 18:28:02+00:00,14.0,2016-06-09 17:51:08+00:00,...,NaN,2016-05-03 19:13:00+00:00,2017-07-01 21:01:07+00:00,103.0,NaN,NaN,NaN,NaN,NaN,NaN
1,175,tommorris,34864460,Nexmo/nexmo-python,551057.0,Nexmo,2017-05-03 17:42:11+00:00,2017-06-06 10:33:03+00:00,4.0,2017-05-26 21:46:43+00:00,...,NaN,2017-05-18 13:22:19+00:00,2017-05-25 16:16:58+00:00,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,260,maddox,124170980,fancybits/pychannels,16461035.0,fancybits,2018-03-07 03:04:43+00:00,2018-03-07 03:11:41+00:00,3.0,NaN,...,NaN,2018-03-07 03:07:19+00:00,2018-03-07 05:25:11+00:00,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,598,jmhodges,26516210,letsencrypt/letsencrypt,9289019.0,letsencrypt,2015-07-22 23:59:26+00:00,2015-11-13 20:39:40+00:00,9.0,2015-11-09 22:30:16+00:00,...,NaN,2015-07-29 22:08:25+00:00,2015-07-29 22:27:19+00:00,2.0,NaN,NaN,NaN,2015-07-29 21:53:06+00:00,2015-08-25 19:22:11+00:00,2.0
4,658,stephencelis,527040,recurly/recurly-client-python,144605.0,recurly,2011-10-28 21:55:09+00:00,2011-10-28 21:55:09+00:00,1.0,2011-10-31 22:27:12+00:00,...,NaN,2011-10-31 22:26:57+00:00,2011-12-03 00:30:45+00:00,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310228,42841451,nenadnikolicai,102973646,fastai/fastai,20547620.0,fastai,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-30 14:37:26+00:00,2018-08-30 14:37:26+00:00,1.0
310229,42848579,Dzyn641,3470471,FortAwesome/Font-Awesome,1505683.0,FortAwesome,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-31 02:30:50+00:00,2018-08-31 02:30:50+00:00,1.0
310230,42848941,ManuBonachela,142422739,google/dopamine,1342004.0,google,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-30 20:00:06+00:00,2018-08-30 20:00:06+00:00,1.0
310231,42872341,BinaryBlockChain,96364578,ethereum/eth-tester,6250754.0,ethereum,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-31 15:44:23+00:00,2018-08-31 15:44:23+00:00,1.0


In [13]:
df_actors_fork

,actor_id,actor_login,type
0,124.0,brosner,general contributor
1,175.0,tommorris,general contributor
2,260.0,maddox,general contributor
3,598.0,jmhodges,general contributor
4,658.0,stephencelis,general contributor
...,...,...,...
111327,NaN,aveuiller,general contributor
111345,NaN,caithagoras0,general contributor
111422,NaN,pastamaker[bot],general contributor
111431,NaN,houssemkouki,general contributor


In [14]:
df_watch = readFile('watchEvent')
df_merged_watch = pd.merge(df_merged_fork, df_watch, how = 'outer')

In [15]:
df_merged_watch

,actor_id,actor_login,repo_id,repo_name,org_id,org_login,earliest_createEvent,latest_createEvent,count_createEvent,earliest_deleteEvent,...,count_pushEvent,earliest_releaseEvent,latest_releaseEvent,count_releaseEvent,earliest_forkEvent,latest_forkEvent,count_forkEvent,earliest_watchEvent,latest_watchEvent,count_watchEvent
0,124.0,brosner,39740335,kelproject/pykube,17834534.0,kelproject,2016-05-21 16:11:14+00:00,2017-03-06 18:28:02+00:00,14.0,2016-06-09 17:51:08+00:00,...,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,175.0,tommorris,34864460,Nexmo/nexmo-python,551057.0,Nexmo,2017-05-03 17:42:11+00:00,2017-06-06 10:33:03+00:00,4.0,2017-05-26 21:46:43+00:00,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-12 14:50:43+00:00,2017-04-12 14:50:43+00:00,1.0
2,260.0,maddox,124170980,fancybits/pychannels,16461035.0,fancybits,2018-03-07 03:04:43+00:00,2018-03-07 03:11:41+00:00,3.0,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-29 07:25:17+00:00,2018-03-29 07:25:17+00:00,1.0
3,598.0,jmhodges,26516210,letsencrypt/letsencrypt,9289019.0,letsencrypt,2015-07-22 23:59:26+00:00,2015-11-13 20:39:40+00:00,9.0,2015-11-09 22:30:16+00:00,...,2.0,NaN,NaN,NaN,2015-07-29 21:53:06+00:00,2015-08-25 19:22:11+00:00,2.0,NaN,NaN,NaN
4,658.0,stephencelis,527040,recurly/recurly-client-python,144605.0,recurly,2011-10-28 21:55:09+00:00,2011-10-28 21:55:09+00:00,1.0,2011-10-31 22:27:12+00:00,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933987,42854098.0,nilbibi,142422739,google/dopamine,1342004.0,google,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-31 01:31:57+00:00,2018-08-31 01:31:57+00:00,1.0
933988,42861855.0,mbulek,142422739,google/dopamine,1342004.0,google,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-31 08:16:10+00:00,2018-08-31 08:16:10+00:00,1.0
933989,42869196.0,ADugeniuc,73328905,pytorch/vision,21003710.0,pytorch,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-31 13:33:14+00:00,2018-08-31 13:33:14+00:00,1.0
933990,42875880.0,baznunes,142422739,google/dopamine,1342004.0,google,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-31 18:38:49+00:00,2018-08-31 18:38:49+00:00,2.0
